In [123]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import datetime
import tensorflow
from tensorflow.keras.models import load_model

In [124]:
from tensorflow.keras.models import load_model
import pickle

# Load the trained Keras model (assuming it's a .h5 file)
model = load_model('model.h5')

# Load the OneHotEncoder for 'Geography'
with open('geography_encoded.pkl', 'rb') as file:
    g_geography_encoded = pickle.load(file)

# Load the LabelEncoder for 'Gender'
with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gen = pickle.load(file)

# Load the scaler (StandardScaler, MinMaxScaler, etc.)
with open("scaler.pkl", 'rb') as file:
    scaler = pickle.load(file)

# Check if all models and encoders have loaded correctly
print(type(model))
print(type(g_geography_encoded))
print(type(label_encoder_gen))
print(type(scaler))


<class 'keras.src.engine.sequential.Sequential'>
<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
<class 'sklearn.preprocessing._label.LabelEncoder'>
<class 'sklearn.preprocessing._data.StandardScaler'>


In [125]:
#Example Input Data

input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':6000,
    'NumOfProducts':2,
    'HasCrCard':True,
    'IsActiveMember':True,
    'EstimatedSalary' : 50000
}

In [126]:
# Make sure 'g_geography_encoded' is an instance of OneHotEncoder, not csr_matrix
geo_encoded = g_geography_encoded.transform([[input_data['Geography']]]).toarray()

# Convert to a DataFrame
geo_encoded_df = pd.DataFrame(geo_encoded, columns=g_geography_encoded.get_feature_names_out(['Geography']))
geo_encoded_df


/home/aryaman/Downloads/GenAI/ANN project/myenv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [127]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,6000,2,True,True,50000


In [128]:
##Encode categorical variable
input_df['Gender']=label_encoder_gen.transform(input_df['Gender'])

In [129]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,6000,2,True,True,50000


In [130]:
##concatenation one hot encoded
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [131]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,6000,2,True,True,50000,1.0,0.0,0.0


In [132]:
##scailing the data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [133]:
#Predict churn
prediction=model.predict(input_scaled)
print(prediction)

1/1 [==============================] - 0s 256ms/step
[[0.01286716]]


In [136]:
prediction_proba=prediction[0][0]
print(prediction_proba)

0.012867162


In [139]:
if prediction_proba >0.5 :
    print("yes the model will churn ")

else:
    print("No,the model will not churn")

No,the model will not churn
